### 서울 격자 데이터 및 각 격자에 포함된 시설물 데이터 전처리 과정

#### 사용된 데이터
+ 전체 격자 코드(gid) 파일 1
+ 각 격자별 행정동 파일 1
+ 격자에 포함된 분류별 시설 파일 3

#### 전처리 내용
+ gid를 기준으로 하나로 합치기 (중복 제거)
+ 결측치는 0으로 채우기
+ 각 시설을 분류별로 나누어 코드 지정하기

---

In [1]:
# 0. 라이브러리 임포트

import numpy as np
import pandas as pd

In [2]:
# 1. 데이터 파일 불러오기

# 격자(gid)
gids = pd.read_csv("./data/Grid_Facilities/격자.csv")

# 행정동
grid_dong = pd.read_csv("./data/Grid_Facilities/격자_행정동.csv", header = None, names = ["gid", "행정동"], skiprows = 1)

# 시설물
facility1 = pd.read_csv("./data/Grid_Facilities/공공_문화체육.csv", header = None, names = ["공공_문화체육", "gid"], skiprows = 1)
facility2 = pd.read_csv("./data/Grid_Facilities/유통_공급시설.csv", header = None, names = ["유통_공급시설", "gid"], skiprows = 1)
facility3 = pd.read_csv("./data/Grid_Facilities/환경기초시설.csv", header = None, names = ["환경기초시설", "gid"], skiprows = 1)

In [3]:
# 2. 무의미한 열 제거 및 gid로 정렬하기

gids = pd.DataFrame(gids["gid"].sort_values(ignore_index = True))
grid_dong.sort_values(by = "gid", ignore_index = True, inplace = True)

In [4]:
# 3. 시설물 데이터의 오탈자 수정, 시설 분류 추가하기

# 3-1. 공공_문화체육
facility1.replace("-", np.nan, inplace = True)
facility1.replace("", np.nan, inplace = True)
facility1.replace("신정119안전센터", "신정119안전센터 (소방서)", inplace = True)
facility1.replace("서울남부여성발전센터", "서울남부여성발전센터 (복지센터)", inplace = True)
facility1.replace("서울교원연수원", "서울교원연수원 (교육청 산하)", inplace = True)
facility1.replace("총포화학협회", "총포화학협회 (경찰청 산하)", inplace = True)
facility1.replace("송파건문소", "송파검문소", inplace = True)
facility1.replace("학교(증산초)", "증산초등학교", inplace = True)
facility1.replace("신방학중학", "신방학중학교", inplace = True)
facility1.replace("경기기계공", "경기기계공업고등학교", inplace = True)
facility1.replace("성동여실고", "성동여실고등학교", inplace = True)
facility1.replace("다니엘학교", "다니엘학교 (특수학교)", inplace = True)
facility1.replace("삼성농아학교", "삼성농아학교 (특수학교)", inplace = True)
facility1.replace("서울정인학교", "서울정인학교 (특수학교)", inplace = True)
facility1.replace("서울정진학교", "서울정진학교 (특수학교)", inplace = True)
facility1.replace("선희농아학교", "선희농아학교 (특수학교)", inplace = True)
facility1.replace("한국육영학교", "한국육영학교 (특수학교)", inplace = True)
facility1.replace("한양대유아교육관,어학관", "한양대학교 유아교육관,어학관", inplace = True)
facility1.replace("동인사마당", "문화시설 (동인사마당)", inplace = True)

# 3-2. 유통_공급시설
facility2.replace("서울쇼핑센타", "서울쇼핑센터", inplace = True)
facility2.replace("롯데쇼핑", "롯데쇼핑센터", inplace = True)

In [5]:
# 4. gid를 기준으로 조인 후, 중복되는 행 제거하기

grid_facilities = pd.merge(gids, grid_dong, on = "gid", how = "inner")
grid_facilities = pd.merge(grid_facilities, facility1, on = "gid", how = "outer")
grid_facilities = pd.merge(grid_facilities, facility2, on = "gid", how = "outer")
grid_facilities = pd.merge(grid_facilities, facility3, on = "gid", how = "outer")

grid_facilities.drop_duplicates()

,gid,행정동,공공_문화체육,유통_공급시설,환경기초시설
0,다사350508,공항동,NaN,NaN,NaN
1,다사350509,공항동,NaN,NaN,NaN
2,다사351507,공항동,NaN,NaN,NaN
3,다사351508,공항동,NaN,NaN,NaN
4,다사351509,공항동,NaN,NaN,NaN
...,...,...,...,...,...
76460,다사720495,강일동,NaN,NaN,NaN
76461,다사720496,강일동,NaN,NaN,NaN
76462,다사720499,강일동,NaN,NaN,NaN
76463,다사720501,강일동,NaN,NaN,NaN


In [6]:
# 5. 결측치 처리하기 - 0으로 채움

grid_facilities.replace(np.nan, 0, inplace = True)

# grid_facilities.info()  # 확인용

In [7]:
# 6. csv로 저장하기 1

grid_facilities.to_csv("./data/Grid_Facilities/grid_facilities_name.csv")

In [7]:
# 7. 단위 격자에 시설물이 존재하면 1, 없으면 0으로 매핑하기

def change_name_to_code(facility_name): 
    if facility_name == 0:
        return 0
    else: 
        return 1

####################

# 복사본
grid_facilities_code = grid_facilities

grid_facilities_code["공공_문화체육"] = grid_facilities["공공_문화체육"].apply(change_name_to_code)
grid_facilities_code["유통_공급시설"] = grid_facilities["유통_공급시설"].apply(change_name_to_code)
grid_facilities_code["환경기초시설"] = grid_facilities["환경기초시설"].apply(change_name_to_code)

grid_facilities_code.info()  # 확인용

print("\t")

# 원본 복구
grid_facilities = pd.read_csv("./data/Grid_Facilities/grid_facilities_name.csv")

grid_facilities.info()   # 확인용

<class 'pandas.core.frame.DataFrame'>
Int64Index: 76465 entries, 0 to 76464
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   gid      76465 non-null  object
 1   행정동      76465 non-null  object
 2   공공_문화체육  76465 non-null  int64 
 3   유통_공급시설  76465 non-null  int64 
 4   환경기초시설   76465 non-null  int64 
dtypes: int64(3), object(2)
memory usage: 3.5+ MB
	
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76465 entries, 0 to 76464
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  76465 non-null  int64 
 1   gid         76465 non-null  object
 2   행정동         76465 non-null  object
 3   공공_문화체육     76465 non-null  object
 4   유통_공급시설     76465 non-null  object
 5   환경기초시설      76465 non-null  object
dtypes: int64(1), object(5)
memory usage: 3.5+ MB


In [8]:
# 8. csv로 저장하기 2

grid_facilities_code.to_csv("./data/Grid_Facilities/grid_facilities_code.csv")